## Wordle
words from https://github.com/dwyl/english-words    
letter freqs from #https://www3.nd.edu/~busiforc/handouts/cryptography/letterfrequencies.html

In [1]:
import numpy as np
import random 
import json

from wordleoracle import WordleOracle ,get_possible_words
colourmap = {0:'grey',1:"yellow",2:"green"}

In [2]:
Orc = WordleOracle()

In [3]:
# quick testing - if not all positions true then hasn't worked
Orc.set_word("abbey")
result = Orc.test_guess("babes")
expected_result = np.array([1,1,2,2,0])
print(result,"==",expected_result,(result==expected_result).all())
Orc.set_word("abbey")
result = Orc.test_guess("babby")
expected_result = np.array([1,1,2,0,2])
print(result,"==",expected_result,(result==expected_result).all())
Orc.set_word("abbey")
result = Orc.test_guess("kebab")
expected_result = np.array([0,1,2,1,1])
print(result,"==",expected_result,(result==expected_result).all())
Orc.set_word("abbey")
result = Orc.test_guess("abbey")
expected_result = np.array([2,2,2,2,2])
print(result,"==",expected_result,(result==expected_result).all())
Orc.set_word("aorta")
result = Orc.test_guess("aaaah",testing=True)
expected_result = np.array([2,1,0,0,0])
print(result,"==",expected_result,(result==expected_result).all())
Orc.set_word("aaabb")
result = Orc.test_guess("bbxbb",testing=True)
expected_result = np.array([0,0,0,2,2])
print(result,"==",expected_result,(result==expected_result).all())
Orc.set_word("aaabb")
result = Orc.test_guess("bbxxb",testing=True)
expected_result = np.array([1,0,0,0,2])
print(result,"==",expected_result,(result==expected_result).all())



[1 1 2 2 0] == [1 1 2 2 0] True
[1 1 2 0 2] == [1 1 2 0 2] True
[0 1 2 1 1] == [0 1 2 1 1] True
[2 2 2 2 2] == [2 2 2 2 2] True
[2 1 0 0 0] == [2 1 0 0 0] True
[0 0 0 2 2] == [0 0 0 2 2] True
[1 0 0 0 2] == [1 0 0 0 2] True


In [29]:
🟨⬛🟩
print("\U0001F642")
print('\u0420\u043e\u0441\u0441\u0438\u044f')
print('\U+1F7E8')

SyntaxError: invalid character '🟨' (U+1F7E8) (<ipython-input-29-218f8271774b>, line 1)

In [4]:
with open('letter_freq.json') as json_file:
    letter_freq_dict = json.load(json_file)

In [21]:
def get_new_possible_words(possible_words,guess_list= [],results_list= []):
    last_result = results_list[-1]
    last_guess = guess_list[-1]
    for i in range(5):
        c = last_guess[i] 
        #print(c,"was",last_result[i])
        if last_result[i] == 0:
            if last_guess.count(c)==1:
                new_possible_words = [word for word in possible_words if not c in word]
            else:
                new_possible_words =possible_words
                pass # TODO
            possible_words = new_possible_words
        if last_result[i] == 1:
            new_possible_words = [word for word in possible_words if c in word]
            possible_words = new_possible_words
            new_possible_words = [word for word in possible_words if  not c is word[i]]
            possible_words = new_possible_words
        if last_result[i] == 2:
            new_possible_words = [word for word in possible_words if c is word[i] ]
            possible_words = new_possible_words
    return possible_words

    
def make_guess_random(possible_words,guess_list,result_list):
    return random.choice(possible_words)

def test_guess(guess):
    secret_word = "crunk"
    if not isinstance(guess, str):
        raise ValueError(guess,"not a string")
    if not len(guess)==5:
        raise ValueError(guess,"not 5 letter word")
    results = np.zeros((5))
    guess = guess.lower()
    for i in range(5): #  not optimised for speed
        if guess[i] == secret_word[i]:
            results[i]=2
        elif guess[i] in secret_word:
            results[i] = 1
#         else: 
#           results[i] = 0
    return results

## Basic random guessing

In [19]:
possible_words = get_possible_words()
guess_list = []
result_list = []
finished = False
i = 0
while(i<6 and finished==False):
    guess = random.choice(possible_words)
    result = test_guess(guess)
    guess_list.append(guess)
    result_list.append(result)
    print("round",i,"| guess was",guess,"| result == ",result)
    print(len(possible_words))
    if (result == 2).all():
        print("Correct!")
        finished = True
    i=i+1
if not finished:
    print("Failed :(")

round 0 | guess was pinyl | result ==  [0. 0. 1. 0. 0.]
15918
round 1 | guess was opelu | result ==  [0. 0. 0. 0. 1.]
15918
round 2 | guess was wevet | result ==  [0. 0. 0. 0. 0.]
15918
round 3 | guess was tumid | result ==  [0. 1. 0. 0. 0.]
15918
round 4 | guess was avoir | result ==  [0. 0. 0. 0. 1.]
15918
round 5 | guess was taker | result ==  [0. 0. 1. 0. 1.]
15918
Failed :(


## Random Guessing (pruning)

In [23]:
rounds = 100
print_vals = False
player_results = np.zeros(rounds)
total_possible_words = get_possible_words()
for k in range(rounds):
    Orc.play()
    possible_words = total_possible_words.copy()
    guess_list = []
    result_list = []
    finished = False
    i = 0
    while(i<27 and not finished):
        guess = random.choice(possible_words)
        result = Orc.test_guess(guess)
        guess_list.append(guess)
        result_list.append(result)
        new_possible_words = get_new_possible_words(possible_words,guess_list,result_list)
        possible_words = new_possible_words
        if print_vals:
            print("round",i,"| guess was",guess,"| possible words remaining",len(possible_words),"result == ",result)
        if (result == 2).all():
            player_results[k]=i
            if print_vals:
                print("Correct!")
            finished = True
        i=i+1
print("finished, average number of rounds was",np.mean(player_results))

finished, average number of rounds was 4.43


## Random Guessing (crane)

In [7]:
rounds = 100
player_results = np.zeros(rounds)
for k in range(rounds):
    Orc.play()
    possible_words = get_possible_words()
    guess_list = []
    result_list = []
    finished = False
    i = 0
    while(i<27 and not finished):
        if i ==0:
            guess = "crane"
        else:
            guess = random.choice(possible_words)
        result = Orc.test_guess(guess)
        guess_list.append(guess)
        result_list.append(result)
        new_possible_words = get_new_possible_words(possible_words,guess_list,result_list)
        possible_words = new_possible_words
        #print("round",i,"| guess was",guess,"| possible words remaining",len(possible_words),"result == ",result)
        if (result == 2).all():
            player_results[k]=i
            #print("Correct!")
            finished = True
        i=i+1
    if not finished: # impossible to reach
        player_results[k]= np.inf
        #print("Failed :( , word was ",Orc.get_word())
print("finished, average number of rounds was",np.mean(player_results))

finished, average number of rounds was 5.307


## weighted guess (too slow doesn't work)

In [13]:
def make_guess_weighted(possible_words): #  too slow
    weights=[]
    for choice in possible_words:
        weight = 0
        for letter, freq in letter_freq_dict.items():
            if letter in choice:
                weight+= freq
        weights.append(weight) 
        weights = [weight/ sum(weights) for weight in weights]

    guesses = random.choices(population = possible_words,weights=weights)

rounds = 1
player_results = np.zeros(rounds)
for k in range(rounds):
    Orc.play()
    possible_words = get_possible_words()
    guess_list = []
    result_list = []
    finished = False
    i = 0
    while(i<27 and not finished):
        guess = make_guess_weighted(possible_words)
        result = Orc.test_guess(guess)
        guess_list.append(guess)
        result_list.append(result)
        new_possible_words = get_new_possible_words(possible_words,guess_list,result_list)
        possible_words = new_possible_words
        #print("round",i,"| guess was",guess,"| possible words remaining",len(possible_words),"result == ",result)
        if (result == 2).all():
            player_results[k]=i
            #print("Correct!")
            finished = True
        i=i+1
    if not finished: # impossible to reach
        player_results[k]= np.inf
        #print("Failed :( , word was ",Orc.get_word())
print("finished, average number of rounds was",np.mean(player_results))

KeyboardInterrupt: 